In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
SELECT * RENAME("Longitude" AS LON, "Latitude" as LAT) FROM NORTHERN_TRAINS_STATION_DATA.TESTING."StationLatLong"

In [ ]:
st.map(cell2.to_df())

In [ ]:
select st_envelope(st_collect( st_makepoint("Longitude","Latitude")) ) BOUNDARY from NORTHERN_TRAINS_STATION_DATA.TESTING."StationLatLong"

In [ ]:
select 
st_envelope(st_collect( st_makepoint("Longitude","Latitude")) ) BOUNDARY, 
ST_CENTROID(BOUNDARY) CENTROID,
ST_X(CENTROID) LON,
ST_Y(CENTROID) LAT
from NORTHERN_TRAINS_STATION_DATA.TESTING."StationLatLong"

In [ ]:
import json
import pydeck as pdk
#find the centre point so the map will render from that location


#create LON and LAT variables -- here we are passing our SQL to the streamlit app

centrepd = cell5.to_df().to_pandas()
LON = centrepd.LON.iloc[0]
LAT = centrepd.LAT.iloc[0]

### transform the data in pandas so the pydeck visualisation tool can view it as a polygon

envelopepd = centrepd
envelopepd["coordinates"] = centrepd["BOUNDARY"].apply(lambda row: json.loads(row)["coordinates"][0])





####visualise on a map

#### create a layer - this layer will visualise the rectangle

polygon_layer = pdk.Layer(
            "PolygonLayer",
            envelopepd,
            opacity=0.3,
            get_polygon="coordinates",
            filled=True,
            get_fill_color=[16, 14, 40],
            auto_highlight=True,
            pickable=False,
        )

 
#### render the map 
    
st.pydeck_chart(pdk.Deck(
    map_style=None,
    initial_view_state=pdk.ViewState(
        latitude=LAT,
        longitude=LON,
        zoom=5,
        height=400
        ),
    
layers= [polygon_layer]

))

In [ ]:
select * from OVERTURE_MAPS__PLACES.CARTO.PLACE 
where ADDRESSES:list[0]:element:country = 'GB' limit 3

In [ ]:
select 
NAMES:primary::text NAME,
PHONES:list[0]:element::text PHONE,
CATEGORIES:main::text CATEGORY,
CATEGORIES:alternate:list[0]['element']::TEXT ALTERNATE,
WEBSITES:list[0]['element']::TEXT WEBSITE,
GEOMETRY

from 

OVERTURE_MAPS__PLACES.CARTO.PLACE 
where ADDRESSES:list[0]:element:country = 'GB' and CATEGORY = 'train_station'

In [ ]:
select * from {{cell8}} A
inner join {{cell5}} B on

ST_WITHIN(TO_GEOGRAPHY(A.GEOMETRY),TO_GEOGRAPHY(B.BOUNDARY))

In [ ]:
with 

places as 
(
select 
NAMES:primary::text NAME,
PHONES:list[0]:element::text PHONE,
CATEGORIES:main::text CATEGORY,
CATEGORIES:alternate:list[0]['element']::TEXT ALTERNATE,
WEBSITES:list[0]['element']::TEXT WEBSITE,
GEOMETRY

from 

OVERTURE_MAPS__PLACES.CARTO.PLACE 
where ADDRESSES:list[0]:element:country = 'GB' and CATEGORY = 'restaurant'),

boundary as (select st_envelope(st_collect( st_makepoint("Longitude","Latitude")) )BOUNDARY from NORTHERN_TRAINS_STATION_DATA.TESTING."StationLatLong")


select places.* , ST_X(GEOMETRY) LON, ST_Y(GEOMETRY) LAT  from places inner join boundary on  ST_WITHIN(places.GEOMETRY, boundary.BOUNDARY)

In [ ]:
placespd = cell10.to_pandas()

poi_l = pdk.Layer(
            'ScatterplotLayer',
            data=placespd,
            get_position='[LON, LAT]',
            get_color='[255,255,255]',
            get_radius=600,
            pickable=True)

#### render the map showing trainstations based on overture maps
    
st.pydeck_chart(pdk.Deck(
    map_style=None,
    initial_view_state=pdk.ViewState(
        latitude=LAT,
        longitude=LON,
        zoom=5,
        height=400
        ),
    
layers= [polygon_layer, poi_l], tooltip = {'text':" Name: {NAME}, alternate: {ALTERNATE}"}

))

In [ ]:
SELECT * RENAME("Longitude" AS LON, "Latitude" as LAT, "CrsCode" as NAME, "Postcode" as ALTERNATE) FROM NORTHERN_TRAINS_STATION_DATA.TESTING."StationLatLong"

In [ ]:
trains_latlon_renamed_pd = cell12.to_pandas()


nw_trains_l = pdk.Layer(
            'ScatterplotLayer',
            data=trains_latlon_renamed_pd,
            get_position='[LON, LAT]',
            get_color='[0,187,2]',
            get_radius=600,
            pickable=True)

#### render the map showing trainstations based on overture maps
    
st.pydeck_chart(pdk.Deck(
    map_style=None,
    initial_view_state=pdk.ViewState(
        latitude=LAT,
        longitude=LON,
        zoom=5,
        height=400
        ),
    
layers= [polygon_layer, poi_l, nw_trains_l], tooltip = {'text':"Name: {NAME}, alternate: {ALTERNATE}"}

))

In [ ]:
SELECT * FROM NORTHERN_TRAINS_STATION_DATA.TESTING."STATION ATTRIBUTES 1" LIMIT 4

In [ ]:
CREATE OR REPLACE TABLE DATA.TRAIN_STATION_INFORMATION AS

SELECT "CRS Code", 
MP, 
"Political Party", 
SNOWFLAKE.CORTEX.COMPLETE('snowflake-arctic',concat(QUESTION,DATA,CONSTRAINTS)) ALTERNATE

FROM 

(

SELECT *,

OBJECT_CONSTRUCT(

'CRS Code',"CRS Code",
'Full Timetable Calls',
"Dec21 Weekday Full Timetable Daily Calls"::INT,
'Emergency Timetable Calls',
"Dec21 Weekday Emergency Timetable Daily Calls"::INT,
'Footfall',
"ORR Station Footfall 2020-21"::INT,
'Parking',
"Car Parking - Free/Chargeable",
'MP',
MP,
'Political Party',
"Political Party",
'Car Parking Spaces',
"Car Parking Spaces"::INT,
'Staffed?',
"Staffed?"

)::TEXT DATA,
'In less than 200 words, write a summary based on the following information' QUESTION,
'Do not include Based on the provided information' CONSTRAINTS
FROM

(SELECT * FROM NORTHERN_TRAINS_STATION_DATA.TESTING."STATION ATTRIBUTES 1")

)

In [ ]:
SELECT * FROM DATA.TRAIN_STATION_INFORMATION

In [ ]:
SELECT * RENAME("Longitude" AS LON, "Latitude" as LAT, "CrsCode" as NAME) FROM NORTHERN_TRAINS_STATION_DATA.TESTING."StationLatLong" A INNER JOIN

DATA.TRAIN_STATION_INFORMATION B ON A."CrsCode" = B."CRS Code" 

In [ ]:
trains_latlon_renamed_pd = cell17.to_pandas()

nw_trains_l = pdk.Layer(
            'ScatterplotLayer',
            data=trains_latlon_renamed_pd,
            get_position='[LON, LAT]',
            get_color='[0,187,2]',
            get_radius=600,
            pickable=True)

#### render the map showing trainstations based on overture maps

tooltip = {
   "html": """<b>Name:</b> {NAME} <br> <b>Alternate:</b> {ALTERNATE}""",
   "style": {
       "width":"50%",
        "backgroundColor": "steelblue",
        "color": "white",
       "text-wrap": "balance"
   }
}
    
st.pydeck_chart(pdk.Deck(
    map_style=None,
    initial_view_state=pdk.ViewState(
        latitude=LAT,
        longitude=LON,
        zoom=5,
        height=700
        ),
    
layers= [polygon_layer, poi_l, nw_trains_l], tooltip = tooltip

))

In [ ]:
CREATE OR REPLACE TABLE DATA.EVENTS_IN_THE_NORTH AS 

SELECT

'''{"DATE":"YYYY-MM-DD", "NAME":"event",DESCRIPTION:"describe what the event is" "CENTROID":{
  "coordinates": [
    0,
    0
  ],
  "type": "Point"
},"COLOR":"Random bright and unique color in RGB presented in an array"}''' JSON,

'''Retrieve 6 events with unique locations the north of england which will happen in 2024.  do not include commentary or notes retrive this in the following json format''' QUESTION,
'ONLY RETURN JSON' CONSTRAINTS,

SNOWFLAKE.CORTEX.COMPLETE('snowflake-arctic',concat(QUESTION,json,CONSTRAINTS)) EVENT_DATA

In [ ]:
select 
H3_POINT_TO_CELL_STRING(TO_GEOGRAPHY(VALUE:CENTROID),5) H3,
VALUE:CENTROID CENTROID,
VALUE:COLOR[0] R,
VALUE:COLOR[1] G,
VALUE:COLOR[2] B,
VALUE:DATE::DATE DATE,
VALUE:NAME::TEXT NAME,
VALUE:DESCRIPTION::TEXT ALTERNATE

from data.events_in_the_north,
lateral flatten(PARSE_JSON(EVENT_DATA))

In [ ]:
events = cell20.to_df()

eventspd = events.group_by('H3','NAME','ALTERNATE','R','G','B').count().to_pandas()


st.write(eventspd)

h3_events = pdk.Layer(
        "H3HexagonLayer",
        eventspd,
        pickable=True,
        stroked=True,
        filled=True,
        extruded=False,
        get_hexagon="H3",
        get_fill_color=["255-R","255-G","255-B"],
        line_width_min_pixels=2,
        opacity=0.4)

#### render the map showing trainstations based on overture maps

tooltip = {
   "html": """<b>Name:</b> {NAME} <br> <b>Alternate:</b> {ALTERNATE}""",
   "style": {
       "width":"50%",
        "backgroundColor": "steelblue",
        "color": "white",
       "text-wrap": "balance"
   }
}

st.pydeck_chart(pdk.Deck(
    map_style=None,
    initial_view_state=pdk.ViewState(
        latitude=LAT,
        longitude=LON,
        zoom=5,
        height=600
        ),
    
layers= [polygon_layer, poi_l, h3_events,nw_trains_l, ], tooltip = tooltip

))

In [ ]:
SELECT * FROM {{cell17}} A

INNER JOIN {{cell20}} B ON H3_LATLNG_TO_CELL_STRING(A.LAT,A.LON,5) = B.H3

In [ ]:
SELECT *, ST_DISTANCE(TO_GEOGRAPHY(CENTROID),TO_GEOGRAPHY(GEOMETRY)) DISTANCE_FROM_EVENT FROM 

(

SELECT * FROM {{cell10}} A

INNER JOIN {{cell20}} B ON H3_LATLNG_TO_CELL_STRING(A.LAT,A.LON,5) = B.H3) where distance_from_event <1000

In [ ]:
SELECT A.MP,A.H3, ARRAY_AGG(

OBJECT_CONSTRUCT('NAME',NAME,'PHONE',PHONE,'WEBSITE',WEBSITE)) WITHIN GROUP ( ORDER BY A.MP ) RESTURANTS


FROM 
(select DISTINCT MP, H3 from {{cell22}}) A

INNER JOIN {{cell23}} B ON A.H3 = B.H3

GROUP BY ALL

In [ ]:
SELECT A.MP,A.H3, ARRAY_AGG(

OBJECT_CONSTRUCT('EVENT',NAME,'DATE',DATE)) WITHIN GROUP ( ORDER BY A.MP ) EVENTS


FROM 
(select DISTINCT MP, H3 from {{cell22}}) A

INNER JOIN {{cell20}} B ON A.H3 = B.H3

GROUP BY ALL

In [ ]:
SELECT A.MP,A.H3, ARRAY_AGG(

OBJECT_CONSTRUCT('Train Station Information',ALTERNATE)) WITHIN GROUP ( ORDER BY A.MP ) TRAIN_STATIONS


FROM 
(select DISTINCT MP, H3 from {{cell22}}) A

INNER JOIN {{cell20}} B ON A.H3 = B.H3

GROUP BY ALL

In [ ]:
CREATE OR REPLACE TABLE DATA.EVENTS_AND_WHAT_IS_AFFFECTED AS 

SELECT A.*, B.EVENTS, C.TRAIN_STATIONS FROM 

{{cell24}} A 

INNER JOIN {{cell25}} B ON A.MP = B.MP

INNER JOIN {{cell26}} C ON A.MP = C.MP

In [ ]:
SELECT * EXCLUDE RESTAURANTS, ARRAY_SLICE(RESTAURANTS,0,10) RESTAURANTS FROM DATA.EVENTS_AND_WHAT_IS_AFFECTED

In [ ]:
CREATE OR REPLACE TABLE DATA.LETTERS_TO_MP AS 

SELECT MP, concat(
'Write to this MP: ',
MP,
'About these events: ',
EVENTS::TEXT,
'Affecting these train stations:',
TRAIN_STATIONS::TEXT,
'And these Restaurants: ',
RESTAURANTS::TEXT,
'The letter is written by Becky - a concerned Citizen'

) PROMPT

, SNOWFLAKE.CORTEX.COMPLETE('mixtral-8x7b',PROMPT) LETTER

FROM {{cell28}}

In [ ]:
SELECT * FROM DATA.LETTERS_TO_MP

In [ ]:
letterspd = cell30.to_pandas()
selected_letter = st.slider('Choose Letter:',0,letterspd.shape[0]-1,1)
st.write(letterspd.LETTER.iloc[selected_letter])